In [49]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import imdb

### Movies dataset

In [61]:
df_movies = pd.read_csv('data/movies.csv')
df_links = pd.read_csv('data/links.csv')
df_movies = pd.merge(df_movies, df_links, how='left', on='movieId')
df_movies.head()

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0


In [90]:
years = []
titles = []
for i in tqdm(range(len(df_movies))):
    year = df_movies['title'][i].replace(' ','')[-5:].replace(')','')
    if str(year).isnumeric() and len(str(year))==4:
        years.append(year)
    else:
        years.append(np.nan)
df_movies['year'] = years

100%|██████████| 58098/58098 [00:00<00:00, 309489.62it/s]


In [97]:
non_year_df = df_movies[df_movies['year'].isna()][['movieId', 'imdbId', 'title', 'year']].reset_index(drop=True)
non_year_df

,movieId,imdbId,title,year
0,32930,428144,Category 6: Day of Destruction,NaN
1,40697,105946,Babylon 5,NaN
2,79607,66079,"Millions Game, The (Das Millionenspiel)",NaN
3,87442,1710542,"Bicycle, Spoon, Apple (Bicicleta, cullera, poma)",NaN
4,98063,87738,Mona and the Time of Burning Love (Mona ja pal...,NaN
...,...,...,...,...
323,192339,2781042,The Sign of Three,NaN
324,192829,8032912,Parwaaz Hai Junoon,NaN
325,193149,7398584,¿Qué te juegas?,NaN
326,193443,6756498,Wall,NaN


In [104]:
ia = imdb.IMDb()
for i in tqdm(range(len(non_year_df))):
    try:
        search = ia.get_movie(str(non_year_df['imdbId'][i]))
        non_year_df.at[i, 'year'] = search['year']
    except Exception as e:
        print(e)
        pass

 41%|████      | 133/328 [05:14<07:52,  2.43s/it]

In [3]:
df_users = pd.read_csv('data/ratings.csv')
df_users.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [4]:
df_tags = pd.read_csv('data/tags.csv')
df_tags.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195
